In [3]:
class SVR_general_cvxopt:
    
    def __init__(self, C = 0.1, epsilon = 0.01, mu = 0.5, kernel = "linear", **kernel_param):
        import numpy as np
        from cvxopt import matrix, solvers, sparse
        from sklearn.metrics.pairwise import pairwise_kernels
        self.sparse = sparse
        self.matrix = matrix
        self.solvers = solvers
        
        self.C = C
        self.epsilon = epsilon
        self.mu = mu
         
        
        self.kernel = kernel
        self.pairwise_kernels = pairwise_kernels
        self.kernel_param = kernel_param
        
        
    def fit(self, X, y):
        # hyperparameters
        C = self.C 
        epsilon =  self.epsilon
        
        kernel = self.kernel
        pairwise_kernels = self.pairwise_kernels
        
        sparse = self.sparse 
        matrix = self.matrix 
        solvers = self.solvers 
        
        # Useful parameters
        ydim = y.shape[0]
        onev = np.ones((ydim,1))
        x0 = np.random.rand(ydim)
        
        def precompute(X, y, kernel):
            # Prematrices for the optimizer
            if kernel == "linear":
                K = pairwise_kernels(X, X, metric = "linear")
            else:
                K = pairwise_kernels(X, X, metric = kernel, **self.kernel_param)
            A = onev.T
            b = 0.0
            G = np.concatenate((np.identity(ydim), -np.identity(ydim)))
            h_ = np.concatenate((100*C*np.ones(ydim)/y, 100*C*np.ones(ydim)/y)); 
            h = h_.reshape(-1, 1)

            # Matrices for the optimizer
            A = matrix(A)
            b = matrix(b)
            G = sparse(matrix(G))
            h = matrix(h)
            Ev = (epsilon*y.T)/100

            # functions for the optimizer
            def obj_func(x):
                return 0.5* x.T @ K @ x + Ev @ (np.abs(x)) - y.T @ x

            def obj_grad(x):
                return x.T @ K + Ev @ (x/np.abs(x)) - y

            def F(x = None, z = None):
                if x is None: return 0, matrix(x0)
                # objective dunction
                val = matrix(obj_func(x))
                # obj. func. gradient
                Df = matrix(obj_grad(x))
                if z is None: return val, Df
                # hessian
                H = matrix(z[0] * K)
                return val, Df, H
            
            return {"F":F, "A":A, "b":b, "G":G, "h":h}
        
        # Solver
        def solver(pre):
            solvers.options['show_progress'] = False
            sol = solvers.cp(F=pre["F"], G=pre["G"], h=pre["h"], A=pre["A"], b=pre["b"])
            return sol

        def get_coefficients(sol, kernel):
            # Support vectors
            beta_1 = np.array(sol['x']).reshape(-1)
            beta_n = np.abs(beta_1)/beta_1.max()
            indx = beta_n > 1e-4
            beta_sv = beta_1[indx]
            x_sv = X[indx,:]
            y_sv = y[indx]

            # get w_phi and b
            
#             other = (kernel, self.kernel_param.values())
#             k_sv = pairwise_kernels(x_sv, x_sv, *other)
            if kernel == "linear":
                k_sv = pairwise_kernels(x_sv, x_sv, metric = "linear")
            else:
                k_sv = pairwise_kernels(x_sv, x_sv, metric = kernel, **self.kernel_param)
                

            cons = np.where(beta_sv >= 0, 1 - epsilon/100, 1 + epsilon/100)

            w_phi = beta_sv @ k_sv
            b = np.mean((y_sv*cons - w_phi))
            
            # keep on memory for latter use
            return beta_sv, x_sv, b
            
            
        def run():
            # get the pre/matrice
            param1 = precompute(X, y, "linear")
            param2 = precompute(X, y, kernel)
            
            # solve for the alphas
            sol1 = solver(param1)
            sol2 = solver(param2)
            
            #compute the coefficients
            coef1 = get_coefficients(sol1, "linear")
            coef2 = get_coefficients(sol2, kernel)
            
            # keep data in memory
            self.coef1 = coef1; self.coef2 = coef2
            return self
        
        return run()
        
    def predict(self, X_):
        mu = self.mu
         
        beta_sv1, x_sv1, b1 = self.coef1
        beta_sv2, x_sv2, b2 = self.coef2
        
        k_test1 = self.pairwise_kernels(x_sv1, X_, metric = "linear")
        
#         other = (self.kernel, self.kernel_param.values())
#         k_test2 = self.pairwise_kernels(x_sv2, X_, *other)
        k_test2 = self.pairwise_kernels(x_sv2, X_, metric = self.kernel, **self.kernel_param)

        w_phi_test1 = beta_sv1 @ k_test1
        w_phi_test2 = beta_sv2 @ k_test2
        
        f1 = w_phi_test1 + b1
        f2 = w_phi_test2 + b2
        
        predict = mu*f1 + (1 - mu)*f2
        return predict
    
    def coef_(self):
        krn = self.kernel
        beta_sv1, _, b1 = self.coef1
        beta_sv2, _, b2 = self.coef2
        return {'sv_linear':beta_sv1, 
                'b_linear':b1, 
                f'sv_{krn}1': beta_sv2,
                f'b_{krn}1':b2}

In [ ]:
class IterativeRun():
    """
    IterativeRun:
        Iterative and ordered SVR prediction and data storage for time series
    
    args: 
        - folder: folder inside Pickle folder to save the resulting predictions
        - n: number of values to predict by iteration
        - itr: number of iterations to run. From last to first. 
                (e.g: n = 2, itr = 10. y_test = -20_-18, ... -2_0)
    
    --Method--
    
        bas_optit: iterate, predict and store data
    
    """
    
    def __init__(self, folder, n = 1, itr = 1):
        self.folder = folder
        self.n = n
        self.itr = itr
        
    def bas_optit(self, C, epsilon, gamma, mu): 
        """
        Iterate, predict and store prediction + error
        
        args: SVR Hyperparameters, and kernel weight (mu*linear_kernel + lmbda*rbf_kernel)
            - C
            - epsilon
            - gamma
            - mu
        """
    #   n: test size, itr: # of iterations
        n = self.n; itr = self.itr

        cas = {}; mape = []; it = itr+1
        yl = len(y)

        # parameters
        hyperparameters = {
            'kernel' : "rbf",
            'C' : C, 
            'epsilon' : epsilon, 
            'mu' : mu,
            'gamma' : gamma, 
        }

        cas[0] = hyperparameters

        itera = np.flip(np.arange(1, it))
        for i in itera:
            j = i*n

            # y, X split
            X_train = X[:yl - j, :]; X_test = X[yl - j : yl - (j-n), :]
            y_train = y[:yl - j];    y_test = y[yl - j : yl - (j-n)]

            # test index
            y_idx = y1.index[yl - j : yl - (j-n)]

            # rescale X and y
            scaler = MaxAbsScaler(); scaler.fit(X_train); 
            X_train = scaler.transform(X_train); X_test = scaler.transform(X_test)

            scaler1 = MaxAbsScaler(); scaler1.fit(y_train)
            y_train = scaler1.transform(y_train).reshape(-1)
            y_test = y_test.reshape(-1)


            # fit and predict
            model = SVR_general_cvxopt(**hyperparameters).fit(X_train, y_train)
            pred = model.predict(X_test)

            # rescale y_test
            y_pred = scaler1.inverse_transform(pred.reshape(-1, 1))

            # keep data
            yi = pd.DataFrame(y_test, index = y_idx, columns = ["real"])
            yi["predict"] = y_pred.reshape(-1)
            yi["resta"] = yi.real - yi.predict
            yi["error"] = (np.abs((yi.real - yi.predict)/yi.real))
            cas[i] = yi
            mape.append(yi.error.mean()*100)

        # Store prediction
        name_ = datetime.now().strftime("%d_%m_%Y_%H_%M_%S")    
        with open(f'Pickles//{self.folder}//{name_}', 'wb') as to_write:
            pickle.dump(cas, to_write)

        mape_mean = sum(mape)/itr

        return -mape_mean